In [1]:
import torch.nn as nn
import math

In [2]:
__all__=['ResNeXt','resnext18','resnext34','resnext50','resnext101','resnext152']

In [29]:
def conv3x3(in_planes,out_planes,stride=1,groups=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes,out_planes,kernel_size=3,stride=stride,groups=1,padding=1,bias=False)

In [30]:
class BasicBlock(nn.Module):
    expansion=1
    def __init__(self,in_planes,planes,stride=1,downsample=None,num_group=32):
        super(BasicBlock,self).__init__()
        self.conv1=conv3x3(in_planes,planes*2,stride)
        self.bn1=nn.BatchNorm2d(planes*2)
        self.relu=nn.ReLU(inplace=True)
        self.conv2=conv3x3(planes*2,planes*2,groups=num_group)
        self.bn2=nn.BatchNorm2d(planes*2)
        self.downsample=downsample
        self.stride=stride
    def forward(self,x):
        residual=x
        out=self.conv1(x)
        out=self.bn1(out)
        out=self.relu(out)
        
        out=self.conv2(out)
        out=self.bn2(out)
        
        if self.downsample is not None:
            residual=self.downsample(x)
        out+=residual
        out=self.relu(out)
        
        return out

In [25]:
class Bottleneck(nn.Module):
    expansion=4
    def __init__(self,in_planes,planes,stride=1,downsample=None,num_group=32):
        super(Bottleneck,self).__init__()
        self.conv1=nn.Conv2d(in_planes,planes*2,kernel_size=1,bias=False)
        self.bn1=nn.BatchNorm2d(planes*2)
        self.conv2=nn.Conv2d(planes*2,planes*2,kernel_size=3,stride=stride,padding=1,bias=False,groups=num_group)
        self.bn2=nn.BatchNorm2d(planes*2)
        self.conv3=nn.Conv2d(planes*2,planes*4,kernel_size=1,bias=False)
        self.bn3=nn.BatchNorm2d(planes*4)
        self.relu=nn.ReLU(inplace=True)
        self.downsample=downsample
        self.stride=stride
    def forward(self,x):
        residual=x
        out=self.conv1(x)
        out=self.bn1(out)
        out=self.relu(out)
        
        out=self.conv2(out)
        out=self.bn2(out)
        out=self.relu(out)
        
        out=self.conv3(out)
        out=self.bn3(out)
        
        if self.downsample is not None:
            residual=self.downsample(x)
        out+=residual
        out=self.relu(out)
        
        return out

In [26]:
class ResNeXt(nn.Module):
    def __init__(self,block,layers,num_classes=1000,num_group=32):
        self.inplanes=64
        super(ResNeXt,self).__init__()
        #这里是原论文的实现，主要是对大尺寸的卷积核尺寸为7
        self.conv1=nn.Conv2d(3,64,kernel_size=7,stride=2,padding=3,bias=False)
        self.bn1=nn.BatchNorm2d(64)
        self.relu=nn.ReLU(inplace=True)
        self.maxpool=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
        self.layer1=self._make_layer(block,64,layers[0],num_group)
        self.layer2=self._make_layer(block,128,layers[1],num_group,stride=2)
        self.layer3=self._make_layer(block,256,layers[2],num_group,stride=2)
        self.layer4=self._make_layer(block,512,layers[3],num_group,stride=2)
        self.avgpool=nn.AvgPool2d(7,stride=1)
        self.fc=nn.Linear(512*block.expansion,num_classes)
        
        for m in self.modules():
            if isinstance(m,nn.Conv2d):
                n=m.kernel_size[0]*m.kernel_size[1]*m.out_channels
                m.weight.data.normal_(0,math.sqrt(2./n))
            elif isinstance(m,nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
    def _make_layer(self,block,planes,blocks,num_group,stride=1):
        downsample=None
        if stride!=1 or self.inplanes!=planes*block.expansion:
            downsample=nn.Sequential(
                nn.Conv2d(self.inplanes,planes*block.expansion,kernel_size=1,
                         stride=stride,bias=False),
                nn.BatchNorm2d(planes*block.expansion),
            )
        layers=[]
        layers.append(block(self.inplanes,planes,stride,downsample,num_group=num_group))
        self.inplanes=planes*block.expansion
        for i in range(1,blocks):
            layers.append(block(self.inplanes,planes,num_group=num_group))
        return nn.Sequential(*layers)
    def forward(self,x):
        x=self.conv1(x)
        x=self.bn1(x)
        x=self.relu(x)
        x=self.maxpool(x)
        
        x=self.layer1(x)
        x=self.layer2(x)
        x=self.layer3(x)
        x=self.layer4(x)
        
        x=self.avgpool(x)
        x=x.view(x.size(0),-1)
        x=self.fc(x)
        
        return x

In [31]:
def resnext18(**kwargs):
    model=ResNeXt(BasicBlock,[2,2,2,2],**kwargs)
    return model

In [32]:
model=resnext18()

In [33]:
model

ResNeXt(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=

In [20]:
def resnext50(**kwargs):
    """Constructs a ResNeXt-50 model.
    """
    model = ResNeXt(Bottleneck, [3, 4, 6, 3], **kwargs)
    return model

In [21]:
model1=resnext50()

In [22]:
model1

ResNeXt(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(